In [427]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
pos = POS("./ckip_data") 
ner = NER("./ckip_data") # entity recognition

# # extend dictionary
# word_to_weight = {
#     "緯來體育台": 1,
#     "鬼金棒": 2,
#     "海底撈": 2
# }
# dictionary = construct_dictionary(word_to_weight)
# print(dictionary)



import pandas as pd
import numpy as np
import time
import math
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

[(3, {'鬼金棒': 2.0, '海底撈': 2.0}), (5, {'緯來體育台': 1.0})]


In [70]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

# print(antusd)
# print(word_senti_score)

In [315]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)
#             senti_word = line.split()[0]
#             senti_score = float(line.split()[1])
#             word_senti_score[senti_word] = senti_score
#             print()
    return paramList

print(get_selenium_param('selenium_url'))


[['Ippudo TW 一風堂(台灣)', 'href="/explore/locations/13656102/ippudo-tw/"', 'href="/p/B-WxOwxpv18/"']]


In [326]:
get_selenium_param('selenium_url')

[['花月嵐拉麵 松山車站店',
  'href="/explore/locations/227701777/"',
  'href="/p/B-hSxkCJNrs/"'],
 ['#花月嵐拉麵', 'href="/explore/tags/花月嵐拉麵/"', 'href="/p/B-HaG68n0fV/"']]

In [468]:
#====== extend senti. dict ======
def slef_extend_senti_dict(word_senti_score):
    with open("dataset/extend_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def slef_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/extend_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        

slef_extend_senti_dict(word_senti_score)
slef_extend_senti_dict_hotpot(word_senti_score)
word_senti_score['good'] = 0.0550611
word_senti_score['棒'] = 0.1
word_senti_score['中規中矩'] = -0.05
word_senti_score['感覺'] = 0.02
word_senti_score['覺得'] = 0.02
word_senti_score['第一'] = 0.2
word_senti_score['虛'] = -0.2
word_senti_score['爛'] = -0.2
word_senti_score['差'] = -0.2
word_senti_score['好差'] = -0.2
word_senti_score['久'] = -0.1
word_senti_score['太久'] = -0.4
word_senti_score['厲害'] = 0.2
word_senti_score['熱死'] = -0.2
word_senti_score['排隊'] = -0.1
word_senti_score['不高'] = -0.3
word_senti_score['鹹'] = -0.1
word_senti_score['好鹹'] = -0.2
word_senti_score['太鹹'] = -0.3
word_senti_score['沒'] = -0.1
word_senti_score['普通'] = -0.02
word_senti_score['不'] = -0.05


In [464]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

[['好', '久', '不', '見', '！', '，', '體力', '一', '年', '不如', '一', '年', '，', '#20191101', '，', '服務', '態度', '超優', '的', '，', '海鮮', '😋', ',', '海', '底', '撈圈字', '遊戲', '送', '一', '盤', '肉', ',', '👍👍👍', '撈撈撈', '，', '💋💋💋', ',', '排隊', '排好', '久', ' ', '超', '值得', ',', ' ', '我', '要', '再', '來', '刷', '二', '訪', ' 😋', '，', ' ', '以', '愛', '吃重', '口味', '的', '我', '來', '說', ' ', '麵條', '的', '硬度', '很', '讚', ' ', '乾', ',', ' ', '想', '吃', '很', '膩害', '的', '溏心蛋', ' ', '這', '家', '沾麵', '真的', '很', '優', ',', '湯頭', '也', '很', '濃', ' ', '我', '不', '愛', '沾麵', ' ', '難波萬', '台灣', '美食', '成為', '我', '的', '愛店', '之', '一👍🏻', '好多', '人', '，', '不', '知道', '要', '排', '多久', ',', '麻', '的', '味道', '超級棒', ',', '推推推', ',', '湯頭', '還是', '很', '棒', '，', '豆芽菜', '的', '鑊氣', '沒', '了', '很', '難過', '，', '大概', '一陣子', '不會', '回訪', '了', ',', 'good excelent', ',', '熱死', '第一', '名', ',', '可惜', '還是', '沒', '把', '湯', '喝完', ',', '應該', '很', '普通', ',', '不', '是', '我', '的', '最愛', '。']]


In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [383]:
# #====== data preprocess ======
# def data_preprocess(articles): 
#     # only get string comment & preprocess fans, likes
#     all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
#     for i in range(len(articles['text'])):     # remove '\n'
#         if type(articles['text'][i]) == str:   # remove non-string
#             if articles['text'][i].find('\n'): # if contain '\n', replace with ','
#                 all_comments.append(articles['text'][i].replace('\n','，'))
#             else:
#                 all_comments.append(articles['text'][i])
#             assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
                
#             # get assigned likes
#             if np.isnan(articles['likes'][i]):
#                 likes.append(1)
#             else:
#                 likes.append(articles['likes'][i])
    
#     return all_comments, assigned_index, fans_count, likes


#====== data preprocess ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            if articles['text'][i].find('\n'): # if contain '\n', replace with ','
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []

            # method2
            # get assigned likes
#             print(i)
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            
            
            
            # method1
#             if np.isnan(articles['likes'][i]):
#                 likes.append(1)
#             else:
#                 likes.append(articles['likes'][i])
    
    return all_comments, assigned_index, fans_count, likes

In [150]:
#====== cal senti score ======
def get_senti_score(text):
    ws_results = ws([text]) # word segment
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 
            print(word,score) # print each senti word's senti score
    
    
    print('sent_score:',total_score)
    if total_score >= 1:
        return 1
    elif total_score <= -1:
        return -1
    else:
        return total_score
    
#====== get each comment's score ======
def get_each_comment_score(all_comments):
    all_comments_scores = []
    for single_comment in all_comments:
        print('\ncomment:',single_comment)
        all_comments_scores.append(get_senti_score(single_comment))
        
    return all_comments_scores


In [101]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
            if not np.isnan(weight[i]):
                weighted_score += (weight[i] * each_comment_score[i])
                total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [102]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [381]:
type(articles['likes'][0])

numpy.int64

In [469]:
#====== main function ======
# get articles, followers
fileName = 'hotpot/article_海底撈火鍋'
articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 
# articles   

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments)

# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')

# # get store score
# store_name = ['鬼金棒','雞吉君','山嵐','你回來啦','麵屋壹慶','勝王','五之神製作所','柑橘','麵屋壹の穴','']
# each_store_score = {}
# print('\nstore final score:',get_store_score(articles, all_comments_scores, assigned_index))


comment: 為了妳們倆的喜帖，特地把我拐去臺北，好啦讓妳們有理由約我吃飯的機會...哈哈🤣，距離我們三個一起吃飯因該有四年前了😅
為了 0.15900875
特地 0.11165215
理由 0.0
吃飯 0.0
機會 0.3837311
三 0.005
吃飯 0.0
sent_score: 0.659392

comment: .﻿，為各位獻上「海底撈單人套餐」‼️﻿，﻿，欸對我大前天才吃﻿，但是昨天想吃蝦滑想瘋了........﻿，撐到今天單人套餐時段才衝來﻿，單人套餐很划算耶399有菜有肉還有海鮮﻿，（飲料喝到飽也含在裡面了）﻿，﻿，一直都是一個蠻能獨處的人﻿，大學常常一個人去逛夜市﻿，因為我一個禮拜有三、四天想去夜市吃😂但找不到跟我一樣一個禮拜想去三天的🤦🏻‍♀️﻿，一個人逛街看電影什麼的都家常便飯﻿，﻿，其實我到店門口前還在掙扎怕尷尬﻿，但想蝦滑的心勝過一切﻿，不用怕啦～很多人會跟你說話﻿，娃娃一上坐，旁邊都在問你能不能拍😆﻿，（啊！米奇也有一副碗筷！），#海底撈單人套餐#海底撈#薇薇吃什麼🍴
獻上 0.15543455
想 0.005
想 0.005
撐到 0.0445837
划算 0.1
一直 0.2729052
常常 0.147622
禮拜 0.0
三 0.005
想 0.005
不 -0.05
一樣 0.0540008
禮拜 0.0
想 0.005
三 0.005
逛街 0.0
電影 0.0
其實 0.2
掙扎 -0.1
尷尬 -0.1
想 0.005
勝過 0.1273962
不用 -0.2652945
很多 0.28742399999999996
😆 0.1
碗筷 0.0
sent_score: 1.00907195

comment: 海底撈圈字遊戲送一盤肉
送 0.15
sent_score: 0.15

comment: 辣到拉肚子😵
sent_score: 0

comment: ที่ไทยคิวยาวมาก มากินที่นี่ก็ได้!! อยากกินต้องได้กิน!!
sent_score: 0

comment: 草兒帶皮卡丘來吃海底撈！
sent_score: 0

comment: 🌹
sent_score: 0

comment: #棠婭去旅遊 ，休假也要把握時間，海底撈的初體驗，真的是肉麻服務😋，而

生日快樂 0.383642475
不知不覺 -0.06228585
不 -0.05
完美 0.6721689999999999
不能 -0.060934800000000004
缺席 -0.325
一直 0.2729052
長大 0.0
一直 0.2729052
學習 0.3755216
謝謝 0.2974824
親愛 0.7535361999999999
記得 0.2896336
生日 0.0
謝謝 0.2974824
祝福 0.25
sent_score: 3.367057425

comment: ไม่อยากต่อคิวที่ CTW เลยมากินที่ Taipei，，#burnedtaipei
sent_score: 0

comment: ʜᴇʀᴀ x ᴅɪᴘsʏ ，#鬆軟燙 ，一起出去上課共同完成的作品💕，與Dipsy合作無間完成美麗的成品❤️ -，只要手繞乾就可以輕鬆完成✅，是不是很簡單呢❤️ - 🔅會員指定Hera專屬優惠👇🏻，胸上長度‼️，染髮+頭皮隔離2500，燙髮+基礎護髮3000 ✔️另外來店申辦會員指定Hera，第一次消費頭皮養護、護髮也有6折優惠❣️ 🔆線上預約👇🏻👇🏻👇🏻，http://www.myhappy.com.tw/Reservation/114/Hera （如果線上預約後需要更改的話，電話撥打02-87896690會有服務人員幫你更改時間） 📍地址：台北市信義區逸仙路42巷27號，（市政府捷運站2號出口出來左手邊，過馬路後第一個巷口左邊就會看到嘍❤️） #女生 #女生短髮 #女生燙髮 #女生染髮 #燙髮 #染髮 #台北燙髮 #鬆軟燙 #雲朵燙 #日系髮型 #日系燙髮 #燙髮推薦 #中長髮 #中長髮燙 ，#波浪捲 #台北染髮 #市政府 #台北 #台北設計師 #台北髮型師 #美髮師 #美容師 #taipei #taiwan #hair #haircut #hairstyles #haircolor #prem #hairprem
共同 0.131986
完成 0.6080297
作品 0.0
💕 0.1
完成 0.6080297
美麗 0.6160439
只要 0.14611355
可以 0.1551797
輕鬆 0.0
完成 0.6080297
不 -0.05
簡單 0.026052
指定 0.241147600000

sent_score: 0

comment: Me dice alguien hace unos año que voy a Taiwán por trabajo y le digo que se deje la medicación. ，Por otro viaje de curro como este en breve @wilson_sasa @brianvanandel
sent_score: 0

comment: 睽違兩年的聚餐值得紀念一下 ♥️🎉
聚餐 0.1
值得 0.23
紀念 0.0550611
sent_score: 0.38506110000000005

comment: 與西門同事の下班宵夜局🙌🏻 ·，叫車來神快 還好有人想到要拍照 趕緊生出合照紀念 但粉醜，#吃飽想睡ㄌ
還好 0.2
趕緊 0.21057775
紀念 0.0550611
想 0.005
sent_score: 0.47063885000000005

comment: Yeah 百年不見の二課⚡️⚡️終於從世界各地飄回來團聚了! 💜 大家還是一樣正😜，Happy birthday to my dear Shirley & Lucy!🎊❤️ 🎂，#bl2 #bffsforever #taipei
不 -0.05
終於 0.162271
團聚 0.1
還是 0.1
一樣 0.0540008
sent_score: 0.3662718

comment: 開心，，#歐陽嘉雯
開心 0.4341682
sent_score: 0.4341682

comment: 敬！人生！
sent_score: 0

comment: 天冷吃#火鍋 真是一大享受，#熱量 排行榜，記得要計算一下呦！，.，.，〓追踪IG @fitgirl_xiaoyun83教你吃，〓私訊IG @fitgirl_xiaoyun83 教你瘦，.，🔵我們提供的服務，☑️增肌減脂飲食餐點，☑️體態形象免費諮詢，☑️客製化專屬服務，☑️有氧運動免費課程，#只要決定一定行，.，.，#熱量 #火鍋控 #豆皮 #丸子 #豆腐 #蝦餃 #貢丸 #元氣林居 #從小胖得救了 #運動營養 #減重計畫 #體態線條 #健康飲食 #運動女孩 #核心 #馬甲線 #雕塑 #健身 #下半身肥胖 #減脂增肌 #小紜窈窕日誌 #happysara

持續 0.0330536
發燒 -0.2330638
可愛 0.6611292
謝謝 0.2974824
喜歡 0.875
重點 0.1391674
吹乾 0.0
不用 -0.2652945
自然 0.1455102
更多 0.187424
作品 0.0
瀏海 0.0
sent_score: 1.8404085

comment: 呵呵～想念好久的味道🌶，夫妻倆👫來吃～海底撈🥘，謝謝～招待.......鮮干貝🤗，#想念的味道，#夫妻倆👫，#晚餐，#海底撈，#招待鮮干貝
呵呵 0.0661072
久 -0.1
謝謝 0.2974824
招待 0.1350369
味道 0.0
招待 0.1350369
sent_score: 0.5336634

comment: 老爸生日快樂！，#62yr
老爸 0.0
生日快樂 0.383642475
sent_score: 0.383642475

comment: 第一攤
第一 0.2
sent_score: 0.2

comment: So delicious! Today's dinner is hot pot! Looking very fancy. But I feel like we may have orders alittle too much. Ups!，好吃！今天的晚飯是火鍋！真好看。我擔心我們可能點太多菜了。，，#taiwan #taipei #taiwanesefood #hotpot #火鍋 #delicious #greattaste #greatlife #greatfood #foods #foodie #foodlover #food #foodporn #awesome #awesomefood #amazingfood #happy #wife #nofilter #nofilterneeded #好吃 #台北 #台灣 #hungry
好吃 0.37684395
好看 0.3426862
擔心 -0.29020949999999995
可能 0.0
好吃 0.37684395
sent_score: 0.8061646

comment: 撈起來
撈起來 0.1
sent_score: 0.1

comment: N年前在北京品嚐海底撈的畫面仍歷歷在目，托老爸、老媽和弟媳的福，回味一下，依然美

開心 0.4341682
成功 0.5034590999999999
約束 0.0
美味 0.6096725000000001
sent_score: 1.5472998

comment: obligatory fancy hotpot for the boyfriend @vince.png. never tried the one in cupertino, but hotpots better in taiwan just because the ingredients are more fresh. ，haidilao is known for their service as well as their hotpot, and the service was exceptional. tea and snacks while we waited, and after we were seated, hot towels, wiped for our glasses (hot pot fog), aprons, nonstop water refills and plate changes and etc; someone was always checking in and not in the annoying way. ，we ordered almost everything on the menu, not even joking (this is only half the table). all hotpot meats were great quality, even the cheaper items; normally in the states, the cheaper meats are noticeably more tough. highly recommend any of the thinly sliced meats, beef and pork especially. ，hands down the best tasting thing ive eaten so far in taiwan: the A5 miyazaki wagyu beef. no competition because of the way it 

生日快樂 0.383642475
願望 0.44799520000000004
不要 -0.1
sent_score: 0.7316376750000001

comment: Thanks for eveythings baobei❤️
sent_score: 0

comment: 五人小聚 🥰🥰🥰，我們下次見😋
sent_score: 0

comment: Hai Di Lao, Taiwan 😋💕 #หม่าล่าดีงามมากก，ที่นี่ไอติมเป็น Häagen-Dazs แหละ #taipei #foodtrip #day3
sent_score: 0

comment: 萬惡晚餐，我的眉毛已離家出走
出走 -0.40580659999999996
sent_score: -0.40580659999999996

comment: 撈撈撈#信義區美食 #台北火鍋
美食 0.16
sent_score: 0.16

comment: 📍信義。海底撈火鍋，湯有點太鹹，但服務超好也很好吃，，#我用美食紀錄我的生活
鹹 -0.1
服務 0.1
超好 0.4242141
好吃 0.37684395
美食 0.16
sent_score: 0.96105805

comment: 趕在生日月最後一週末慶生，上週感冒延期，這週傳染給我，差點再延期，還好紫錐花茶救了我，兩天痊癒，爽😊，推薦給快要感冒和輕微感冒的人，提升免疫力的功效我見證💪，#美伊祝你生日快樂，#一個大臉一個黑臉
最後 -0.0175984
感冒 -0.06904439999999999
延期 0.0
差點 0.0
延期 0.0
還好 0.2
痊癒 0.0
感冒 -0.06904439999999999
輕微 0.0
感冒 -0.06904439999999999
提升 0.08609
功效 0.2851968
生日快樂 0.383642475
sent_score: 0.730197675

comment: 笑想很久的海底撈😍，今天終於成功征服了🥰，加上與我默契不好的拍攝者，讓我手機增添了許多NG照😂😂😂
久 -0.1
終於 0.162271
成功 0.5034590999999999
征服 0.0
🥰 0.1
加上 0.16797105
不 -0.05
增添 0.0330536


In [ ]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]




In [399]:
# store_names = ['辣麻味噌拉麵 鬼金棒','雞吉君拉麵','山嵐拉麵','你回來啦 Okaeri 拉麵']
# store_page_xpath = ['href="/explore/locations/541098880/"','href="/explore/tags/雞吉君拉麵/"','href="/explore/locations/610378299165578/"','href="/explore/locations/1009572868/okaeri/"']
# first_page_xpath = ['href="/p/B-Rf46-jXl_/"','href="/p/B-CEZjVl8GC/"','href="/p/B-Wy4mhnnvk/"', 'href="/p/B-KNu4oFwcv/"']
# get_page_count = [500,130,500,500]


# store_names = ['一蘭拉麵','麵屋壹の穴 ICHI','勝王']
# store_page_xpath = ['href="/explore/locations/255951018612300/"','href="/explore/locations/1719618628292660/ichi/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"','href="/p/B-ZXrQEnbZ-/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [1200,1000,1000]


# store_names = ['一蘭拉麵']
# store_page_xpath = ['href="/explore/locations/255951018612300/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"']
# get_page_count = [4000]


# store_names = ['柑橘 Shinn','五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/765135860311072/shinn/"','href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZpZrvnZru/"','href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [600,700,700,700]

# store_names = ['五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [700,700,700]

store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B-3_k9AHF81/"']


In [ ]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(4000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(2)
            else:
                break
            
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(2)
        else:
            break

        # stop date = 2019.8.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

In [400]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(4000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '7':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime)

這一鍋皇室秘藏鍋物finished!
1462.405298948288


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
